# Data Preprocessing

In [13]:
# load library
import numpy as np
import pandas as pd
import zipfile
import urllib.request
import io

In [14]:
# load raw data
url = "https://github.com/zxyao5148/STAT3612_2023_1A_GroupProject/raw/main/raw_data/ehr_preprocessed_seq_by_day_cat_embedding.pkl.zip"
with urllib.request.urlopen(url) as response:
    with zipfile.ZipFile(io.BytesIO(response.read())) as zip_file:
        with zip_file.open("ehr_preprocessed_seq_by_day_cat_embedding.pkl", "r") as file:
            EHR = pd.read_pickle(file)
train = pd.read_csv("https://github.com/zxyao5148/STAT3612_2023_1A_GroupProject/raw/main/raw_data/train.csv")
valid = pd.read_csv("https://github.com/zxyao5148/STAT3612_2023_1A_GroupProject/raw/main/raw_data/valid.csv")
test = pd.read_csv("https://github.com/zxyao5148/STAT3612_2023_1A_GroupProject/raw/main/raw_data/test.csv")
print(train.shape)
print(test.shape)
print(valid.shape)
print(len(EHR['feature_cols']))

(55941, 13)
(17933, 12)
(13598, 13)
171


In [15]:
# drop image-related columns
train = train.drop(train.columns[[6,7,8,11]], axis=1)
test = test.drop(test.columns[[6,7,8,11]], axis=1)
valid = valid.drop(valid.columns[[6,7,8,11]], axis=1)
train.head(5)

,id,subject_id,hadm_id,admittime,dischtime,deathtime,StudyDate,StudyTime,readmitted_within_30days
0,10869829_25238191,10869829,25238191,2141-08-20 12:41:00,2141-09-01 13:22:00,NaN,21410828,90151.343,0
1,10869829_25238191,10869829,25238191,2141-08-20 12:41:00,2141-09-01 13:22:00,NaN,21410831,130627.031,0
2,10869829_25238191,10869829,25238191,2141-08-20 12:41:00,2141-09-01 13:22:00,NaN,21410821,80228.937,0
3,17910612_22301530,17910612,22301530,2188-03-04 19:49:00,2188-04-19 00:00:00,2188-04-19 00:00:00,21880405,200636.062,1
4,17910612_22301530,17910612,22301530,2188-03-04 19:49:00,2188-04-19 00:00:00,2188-04-19 00:00:00,21880321,114135.046,1


In [16]:
# extract last-day observations
train['StudyDate'] = pd.to_datetime(train['StudyDate'], format='%Y%m%d')
train_latest_date_idxs = train.groupby('id')['StudyDate'].idxmax()
train_masked = train.loc[train_latest_date_idxs]

test['StudyDate'] = pd.to_datetime(test['StudyDate'], format='%Y%m%d')
test_latest_date_idxs = test.groupby('id')['StudyDate'].idxmax()
test_masked = test.loc[test_latest_date_idxs]

valid['StudyDate'] = pd.to_datetime(valid['StudyDate'], format='%Y%m%d')
valid_latest_date_idxs = valid.groupby('id')['StudyDate'].idxmax()
valid_masked = valid.loc[valid_latest_date_idxs]

print(train_masked.shape)
print(test_masked.shape)
print(valid_masked.shape)

(9271, 9)
(2936, 8)
(2325, 9)


In [17]:
# add two new predictors (length of stay + previous number of admissions)
# length of stay
train_masked['dischtime'] = pd.to_datetime(train_masked['dischtime'])
train_masked['admittime'] = pd.to_datetime(train_masked['admittime'])
train_masked['LoS'] = (train_masked['dischtime'] - train_masked['admittime']).dt.days

test_masked['dischtime'] = pd.to_datetime(test_masked['dischtime'])
test_masked['admittime'] = pd.to_datetime(test_masked['admittime'])
test_masked['LoS'] = (test_masked['dischtime'] - test_masked['admittime']).dt.days

valid_masked['dischtime'] = pd.to_datetime(valid_masked['dischtime'])
valid_masked['admittime'] = pd.to_datetime(valid_masked['admittime'])
valid_masked['LoS'] = (valid_masked['dischtime'] - valid_masked['admittime']).dt.days

# previous number of admissions
train_masked = train_masked.sort_values('admittime')
train_masked['prev_admits'] = train_masked.groupby('subject_id').cumcount()

test_masked = test_masked.sort_values('admittime')
test_masked['prev_admits'] = test_masked.groupby('subject_id').cumcount()

valid_masked = valid_masked.sort_values('admittime')
valid_masked['prev_admits'] = valid_masked.groupby('subject_id').cumcount()

train_masked.head(5)

,id,subject_id,hadm_id,admittime,dischtime,deathtime,StudyDate,StudyTime,readmitted_within_30days,LoS,prev_admits
41856,17195991_23542772,17195991,23542772,2110-01-11 22:47:00,2110-01-18 10:25:00,NaN,2110-01-16,90654.546,0,6,0
24332,13721591_20342223,13721591,20342223,2110-02-09 18:13:00,2110-02-22 20:51:00,NaN,2110-02-19,41948.468,0,13,0
20119,19170541_22178312,19170541,22178312,2110-02-28 21:48:00,2110-03-12 17:47:00,NaN,2110-03-11,81842.812,0,11,0
51567,15554295_27705504,15554295,27705504,2110-03-09 03:54:00,2110-05-18 11:34:00,NaN,2110-05-04,60653.312,0,70,0
29152,17643026_29919541,17643026,29919541,2110-03-25 11:15:00,2110-03-29 17:17:00,NaN,2110-03-28,140521.453,0,4,0


In [18]:
# construct merged dataframe (admit info + feature value)
train_merged_admits_latest = []
train_merged_admits_median = []
for _, admit in train_masked.iterrows():
    feat_arr_latest = pd.Series(EHR['feat_dict'][admit['id']][-1], index=EHR['feature_cols'])
    feat_arr_median = round(pd.Series(np.nanmedian(EHR['feat_dict'][admit['id']], axis=0), index=EHR['feature_cols']))
    train_merged_admits_latest.append(pd.concat([admit, feat_arr_latest]))
    train_merged_admits_median.append(pd.concat([admit, feat_arr_median]))
train_merged_df_latest = pd.DataFrame(train_merged_admits_latest)
train_merged_df_median = pd.DataFrame(train_merged_admits_median)

test_merged_admits_latest = []
test_merged_admits_median = []
for _, admit in test_masked.iterrows():
    feat_arr_latest = pd.Series(EHR['feat_dict'][admit['id']][-1], index=EHR['feature_cols'])
    feat_arr_median = pd.Series(np.nanmedian(EHR['feat_dict'][admit['id']], axis=0), index=EHR['feature_cols'])
    test_merged_admits_latest.append(pd.concat([admit, feat_arr_latest]))
    test_merged_admits_median.append(pd.concat([admit, feat_arr_median]))
test_merged_df_latest = pd.DataFrame(test_merged_admits_latest)
test_merged_df_median = pd.DataFrame(test_merged_admits_median)

valid_merged_admits_latest = []
valid_merged_admits_median = []
for _, admit in valid_masked.iterrows():
    feat_arr_latest = pd.Series(EHR['feat_dict'][admit['id']][-1], index=EHR['feature_cols'])
    feat_arr_median = pd.Series(np.nanmedian(EHR['feat_dict'][admit['id']], axis=0), index=EHR['feature_cols'])
    valid_merged_admits_latest.append(pd.concat([admit, feat_arr_latest]))
    valid_merged_admits_median.append(pd.concat([admit, feat_arr_median]))
valid_merged_df_latest = pd.DataFrame(valid_merged_admits_latest)
valid_merged_df_median = pd.DataFrame(valid_merged_admits_median)

print(train_merged_df_latest.shape)
print(test_merged_df_latest.shape) 
print(valid_merged_df_latest.shape)

(9271, 182)
(2936, 181)
(2325, 182)


In [7]:
# missing value imputation
# first drop features with too much missing values (50% as a threshold)
# then impute the rest with median
feat_arrs = [] # list of feature arrays (flattened)
for id in train_masked['id']:
    feat_arrs.extend(EHR['feat_dict'][id])
feat_mat = np.vstack(feat_arrs)
num_nan = np.isnan(feat_mat).sum(axis=0)
print(num_nan)
# turns out that there is no missing value
# no need for further processing...

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [8]:
# standardization for numeric features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

cat_cols = [EHR['feature_cols'][i] for i in EHR['cat_idxs']]
icd_cols = EHR['icd_cols']
num_cols = list(set(EHR['feature_cols']) - set(cat_cols) - set(icd_cols)) 
num_cols.extend(['LoS', 'prev_admits']) 

train_merged_df_median[num_cols] = scaler.fit_transform(train_merged_df_median[num_cols])
train_merged_df_latest[num_cols] = scaler.fit_transform(train_merged_df_latest[num_cols])
test_merged_df_median[num_cols] = scaler.fit_transform(test_merged_df_median[num_cols])
test_merged_df_latest[num_cols] = scaler.fit_transform(test_merged_df_latest[num_cols])
valid_merged_df_median[num_cols] = scaler.fit_transform(valid_merged_df_median[num_cols])
valid_merged_df_latest[num_cols] = scaler.fit_transform(valid_merged_df_latest[num_cols])

train_merged_df_latest.head(5)

,id,subject_id,hadm_id,admittime,dischtime,deathtime,StudyDate,StudyTime,readmitted_within_30days,LoS,...,PRE-NATAL VITAMINS,ANESTHETICS,ANTIBIOTICS,ANTIHYPERGLYCEMICS,ANTIINFLAM.TUMOR NECROSIS FACTOR INHIBITING AGENTS,SEDATIVE/HYPNOTICS,ANTIDOTES,AUTONOMIC DRUGS,VITAMINS,BIOLOGICALS
0,17195991_23542772,17195991,23542772,2110-01-11 22:47:00,2110-01-18 10:25:00,NaN,2110-01-16,90654.546,0,-0.405473,...,-0.013935,-0.455273,-0.449292,-0.318458,0.0,-0.418357,-0.047707,2.185809,-0.381595,-0.304994
1,13721591_20342223,13721591,20342223,2110-02-09 18:13:00,2110-02-22 20:51:00,NaN,2110-02-19,41948.468,0,-0.026227,...,-0.013935,0.210369,-0.130508,-0.232069,0.0,-0.418357,-0.047707,-0.306970,-0.381595,-0.304994
2,19170541_22178312,19170541,22178312,2110-02-28 21:48:00,2110-03-12 17:47:00,NaN,2110-03-11,81842.812,0,-0.134583,...,-0.013935,0.210369,-0.289900,-0.318458,0.0,-0.418357,-0.047707,-0.306970,-0.381595,-0.304994
3,15554295_27705504,15554295,27705504,2110-03-09 03:54:00,2110-05-18 11:34:00,NaN,2110-05-04,60653.312,0,3.061922,...,-0.013935,0.210369,1.463409,-0.232069,0.0,3.126321,-0.047707,1.354882,-0.381595,-0.304994
4,17643026_29919541,17643026,29919541,2110-03-25 11:15:00,2110-03-29 17:17:00,NaN,2110-03-28,140521.453,0,-0.513830,...,-0.013935,-0.455273,-0.608684,-0.059292,0.0,-0.418357,-0.047707,-0.306970,-0.381595,1.561410


In [9]:
# drop features with constant values in the training set
feat_df_latest = pd.DataFrame(train_merged_admits_latest[9:])
feat_df_median = pd.DataFrame(train_merged_admits_median[9:])
const_cols_latest = [col for col in feat_df_latest.columns if feat_df_latest[col].nunique() <= 1]
const_cols_median = [col for col in feat_df_median.columns if feat_df_median[col].nunique() <= 1]
print(const_cols_latest)
print(const_cols_median)

train_merged_df_latest = train_merged_df_latest.drop(columns=const_cols_latest)
train_merged_df_median = train_merged_df_median.drop(columns=const_cols_median)
test_merged_df_latest = test_merged_df_latest.drop(columns=const_cols_latest)
test_merged_df_median = test_merged_df_median.drop(columns=const_cols_median)
valid_merged_df_latest = valid_merged_df_latest.drop(columns=const_cols_latest)
valid_merged_df_median = valid_merged_df_median.drop(columns=const_cols_median)

print(train_merged_df_latest.shape)
print(test_merged_df_latest.shape)
print(valid_merged_df_latest.shape)

['H30-H36', 'N00-N08', 'C73-C75', 'M60-M63', 'A20-A28', 'M50-M54', 'N25-N29', 'J00-J06', 'M05-M14', 'K70-K77', 'Basophils Other Body Fluid', 'Basophils Pleural', 'Eosinophils Joint Fluid', 'pH Urine', 'Basophils Joint Fluid', 'Basophils Ascites', 'Lymphocytes Joint Fluid', 'ANTIINFLAM.TUMOR NECROSIS FACTOR INHIBITING AGENTS']
['H30-H36', 'N00-N08', 'C73-C75', 'M60-M63', 'A20-A28', 'M50-M54', 'N25-N29', 'J00-J06', 'M05-M14', 'K70-K77', 'Basophils Other Body Fluid', 'Basophils Pleural', 'Eosinophils Joint Fluid', 'Eosinophils Ascites', 'pH Urine', 'Lymphocytes Ascites', 'Eosinophils Other Body Fluid', 'Basophils Joint Fluid', 'H Blood', 'Basophils Ascites', 'Lymphocytes Other Body Fluid', 'Monocytes Ascites', 'Lymphocytes Joint Fluid', 'ANTIINFLAM.TUMOR NECROSIS FACTOR INHIBITING AGENTS']
(9271, 158)
(2936, 157)
(2325, 158)


In [ ]:
# TO BE COMPLETED (SMOTE)
# address class imbalance issue (in training set)
# oversampling the minority class
print(train_merged_df_latest['readmitted_within_30days'].value_counts()) # check class distribution
print(valid_merged_df_latest['readmitted_within_30days'].value_counts())
#print(test_merged_df_latest['readmitted_within_30days'].value_counts())

In [11]:
# write csv files
train_merged_df_latest.to_csv('train_merged_latest_feat.csv', index=False)
train_merged_df_median.to_csv('train_merged_median_feat.csv', index=False)
test_merged_df_latest.to_csv('test_merged_latest_feat.csv', index=False)
test_merged_df_median.to_csv('test_merged_median_feat.csv', index=False)
valid_merged_df_latest.to_csv('valid_merged_latest_feat.csv', index=False)
valid_merged_df_median.to_csv('valid_merged_median_feat.csv', index=False)